In [1]:
import sklearn 
import pandas as pd
import xml.etree.ElementTree as ET


__First of all they did not mentioned what sklearn version they used!!!__


As described in the paper, the first step is to select base classifiers. 
The selected base classifiers are trained with default parameter settings with 10-fold cross-validation.
As input data, the training data set and its ground truth labels, per single modality is used.
For the audio MFCC features, we set NaN values to 0, and calculate the average of each MFCC coefficient over all frames.

# Load input data


# Description:

## Available Data
There are variouse csv files and data files available. It's very messy. 
There is one file called "CoE_dataset_offical_release.zip"! 
We extract this file and use this data included their for now! 

## Meta Data
In the original paper there is no information given what is included in the metadata. 
Looking at the paper describing the data set (Right Inflight? A Dataset for Exploring the Automatic Prediction of Movies Suitable for a Watching Situation
) we found out that as metadata they used language, year published, genre, country, runtime and age rating. We assume, since the author of our paper didn't say otherwise, that they used the same metadata. 

## Visual Data: 
The visual data is provied as a csv file for each movie, containing two rows. According to the paper of the dataset they calculated following visual features, Histogram of Oriented Gradients (HOG) gray, Color Moments, local binary patterns (LBP) and Gray Level Run Length Matrix, but don't say how the csv file represents them. Also as mentioned the csv file just has two rows which would not ad up to the mentioned 4 visual features. __We are treating all values as seperate column!__

## Audio Data: 
Audio features is also provided per movie as a csv file. Each audio feature consits of 12 coefficients for multiple frames.

## Textual Data
The textual data is just one file containing the tdf-idf matrix. The first line are the row names for each word. 
While the columns are the associated movie. __There is no indication to which movie each column belongs! Thus we need to assume this!__

__For now we assume the order is the same as in the df_labled_movies dataframe!!!__



In [2]:

df_labled_movies = pd.read_csv("./data/CoE_dataset/Dev_set/dev_set_groundtruth_and_trailers.csv", sep=';')
del df_labled_movies['trailer']
df_labled_movies = df_labled_movies[['movie','filename', 'goodforairplane']]
display(df_labled_movies.head(10))


### Load Meta Data ###

def load_meta_data( filenames ): 
    
    raw_data = []
    
    for file in filenames: 
        file_path = f'./data/CoE_dataset/Dev_set/XML/{file}.xml'
        with open(file_path) as f: 
            tree = ET.parse(f)
            movie = tree.find('movie')
            
            lang = movie.get('language')
            year = movie.get('year')
            genre = movie.get('genre')
            country = movie.get('country')
            runtime = movie.get('runtime')
            age_rating = movie.get('rated')
             
            raw_data.append( (file,lang,year,genre,country,runtime,age_rating) )
    
    return pd.DataFrame(raw_data, columns=['filename','language','year','genre','country','runtime','rated'])


df_meta_data = load_meta_data( df_labled_movies['filename']  )
display(df_meta_data.head(10))
display(df_meta_data.dtypes)

### Load Visual Data ###

def load_visual_data( filenames ):
    data_list = []
    
    for file in filenames: 
        file_path = f'./data/CoE_dataset/Dev_set/vis_descriptors/{file}.csv'
        df_data = pd.read_csv(file_path,index_col=None, header=None)
        data_list.append(df_data)
        
    return pd.concat(data_list, axis = 0, keys = filenames,names=('filename','vis_data'),  sort=False)

df_visual_data = load_visual_data( df_labled_movies['filename']  )
display(df_visual_data.head(10))

### Load Audio Data ###

def load_audio_data( filenames ):
    data_list = []
    
    for file in filenames: 
        file_path = f'./data/CoE_dataset/Dev_set/audio_descriptors/{file}.csv'
        df_data = pd.read_csv(file_path,index_col=None, header=None)
        data_list.append(df_data)
        
    return pd.concat(data_list, axis = 0, keys = filenames,names=('filename','freq_coeff'),  sort=False)

df_audio_data = load_audio_data( df_labled_movies['filename']  )
display(df_audio_data.head(20))


### Load textual Data ###

def load_text_data(filenames):
    

    data_list = []
    file_path = f'./data/CoE_dataset/Dev_set/text_descriptors/tdf_idf_dev.csv'
    #somehow pandas can not really handle that the first line is row names.(at least I didn't find a better way) 
    # thus we do it a little complicated here
    header_index = pd.read_csv(file_path, index_col=0,nrows=1 ).reset_index().columns
    df_data = pd.read_csv(file_path, header=None, index_col=False,skiprows=1)
    df_data.set_index(header_index, inplace=True)
    df_data.columns = filenames
    return df_data.T #row are should be represented by movie names

df_text_data = load_text_data(df_labled_movies['filename'] )
display(df_text_data.head(20))
display(df_text_data.shape)
display(df_text_data.describe())



,movie,filename,goodforairplane
0,Seventh Son,Seventh_Son,1
1,Welcome to Me,Welcome_to_Me,0
2,The Judge,The_Judge,0
3,Transformers Age of Extinction,Transformers__Age_of_Extinction,0
4,The Normal Heart,The_Normal_Heart,1
5,The Phantom Tollbooth,The_Phantom_Tollbooth,1
6,Andaz Apna Apna,Andaz_Apna_Apna,1
7,Hotel Transylvania,Hotel_Transylvania,1
8,The Matrix,The_Matrix,1
9,Into the Wild,Into_the_Wild,1


,filename,language,year,genre,country,runtime,rated
0,Seventh_Son,English,2014,"Action, Adventure, Fantasy","USA, UK, Canada, China",102 min,PG-13
1,Welcome_to_Me,English,2014,"Comedy, Drama",USA,105 min,R
2,The_Judge,English,2014,Drama,USA,141 min,R
3,Transformers__Age_of_Extinction,English,2014,"Action, Adventure, Sci-Fi","USA, China",165 min,PG-13
4,The_Normal_Heart,English,2014,Drama,USA,132 min,TV-MA
5,The_Phantom_Tollbooth,English,1970,"Family, Adventure, Animation",USA,90 min,G
6,Andaz_Apna_Apna,Hindi,1994,"Comedy, Family, Romance",India,160 min,PG
7,Hotel_Transylvania,English,2012,"Animation, Comedy, Family",USA,91 min,PG
8,The_Matrix,English,1999,"Action, Sci-Fi","USA, Australia",136 min,R
9,Into_the_Wild,"English, Danish",2007,"Adventure, Biography, Drama",USA,148 min,R


filename    object
language    object
year        object
genre       object
country     object
runtime     object
rated       object
dtype: object

0        1        2    \
filename                        vis_data                               
Seventh_Son                     0         0.047044  0.11619  0.13633   
                                1         0.056526  0.12516  0.14628   
Welcome_to_Me                   0         0.307170  0.33422  0.33112   
                                1         0.304660  0.33193  0.33124   
The_Judge                       0         0.000000  0.00000  0.00000   
                                1         0.000000  0.00000  0.00000   
Transformers__Age_of_Extinction 0         0.199960  0.26934  0.27986   
                                1         0.189130  0.25738  0.27465   
The_Normal_Heart                0         0.000000  0.00000  0.00000   
                                1         0.000000  0.00000  0.00000   

                                               3         4         5    \
filename                        vis_data                                 
Seventh_Son                     0         0.066194  0.072554  0.172670   
                                1         0.082497  0.079331  0.175380   
Welcome_to_Me                   0         0.331240  0.311140  0.336440   
                                1         0.331380  0.307880  0.332700   
The_Judge                       0         0.000000  0.000000  0.000000   
                                1         0.000000  0.000000  0.000000   
Transformers__Age_of_Extinction 0         0.237250  0.308440  0.332420   
                                1         0.236640  0.303320  0.329890   
The_Normal_Heart                0         0.000000  0.038749  0.083701   
                                1         0.000000  0.201350  0.297900   

                                              6         7         8    \
filename                        vis_data                                
Seventh_Son                     0         0.21519  0.070574  0.071423   
                                1         0.21839  0.093521  0.074837   
Welcome_to_Me                   0         0.33616  0.344790  0.169830   
                                1         0.33357  0.343050  0.173300   
The_Judge                       0         0.00000  0.000000  0.000000   
                                1         0.00000  0.000000  0.000000   
Transformers__Age_of_Extinction 0         0.32998  0.325000  0.307350   
                                1         0.32888  0.322460  0.305430   
The_Normal_Heart                0         0.10544  0.121500  0.038749   
                                1         0.39682  0.553360  0.201350   

                                               9      ...           816  \
filename                        vis_data              ...                 
Seventh_Son                     0         0.149380    ...        731.69   
                                1         0.150250    ...        689.95   
Welcome_to_Me                   0         0.273790    ...        394.34   
                                1         0.280760    ...        397.26   
The_Judge                       0         0.000000    ...     230400.00   
                                1         0.000000    ...     230400.00   
Transformers__Age_of_Extinction 0         0.334310    ...       1112.60   
                                1         0.335510    ...       1120.60   
The_Normal_Heart                0         0.083701    ...      34463.00   
                                1         0.297900    ...      41786.00   

                                                817        818        819  \
filename                        vis_data                                    
Seventh_Son                     0            502.01   1.897000   2.278800   
                                1            474.97   2.267600   2.588700   
Welcome_to_Me                   0            167.91  20.337000  21.276000   
                                1            168.23  20.426000  21.300000   
The_Judge                       0         119950.00  

0      1      2      3      4      5      6      \
filename      freq_coeff                                                    
Seventh_Son   0             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              1             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              2             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              3             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              4             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              5             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              6             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              7             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              8             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              9             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              10            NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              11            NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              12            NaN    NaN    NaN    NaN    NaN    NaN    NaN   
Welcome_to_Me 0             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              1             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              2             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              3             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              4             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              5             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              6             NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                             7         8         9      ...    23078  23079  \
filename      freq_coeff                                ...                   
Seventh_Son   0          -51.23500 -25.77500 -17.41000  ...      NaN    NaN   
              1            4.76010  10.41400  15.93500  ...      NaN    NaN   
              2           -8.65190  -6.16670  -7.37720  ...      NaN    NaN   
              3           -8.13970  -8.09110 -14.56800  ...      NaN    NaN   
              4           -1.72450   2.19680   1.14500  ...      NaN    NaN   
              5            0.93079   9.88010  15.52800  ...      NaN    NaN   
              6           -2.20740   4.47300   6.56920  ...      NaN    NaN   
              7           -2.63550  -1.67510  -6.13000  ...      NaN    NaN   
              8           -0.33020   1.28230  -0.95568  ...      NaN    NaN   
              9            0.25014   7.69770  11.97200  ...      NaN    NaN   
              10          -1.42530   6.37270  10.30700  ...      NaN    NaN   
              11          -1.03030   0.39305  -1.11160  ...      NaN    NaN   
              12           0.41208  -2.07840  -4.61110  ...      NaN    NaN   
Welcome_to_Me 0                NaN       NaN       NaN  ...      NaN    NaN   
              1                NaN       NaN       NaN  ...      NaN    NaN   
              2                NaN       NaN       NaN  ...      NaN    NaN   
              3                NaN       NaN       NaN  ...      NaN    NaN   
              4                NaN       NaN       NaN  ...      NaN    NaN   
              5                NaN       NaN       NaN  ...      NaN    NaN   
              6                NaN       NaN       NaN  ...      NaN    NaN   

                          23080  23081  23082  23083  23084  23085  23086  \
filename      freq_coeff                                                    
Seventh_Son   0             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              1             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              2             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              3             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              4             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              5             NaN    NaN    NaN    NaN 

,24000,baby,baseball,big,doc,escort,frozen,heroes,high,huck,...,years.1,york,yorks,young,young.1,younger,youngja,zebra,zellweger,zoologists
filename,,,,,,,,,,,,,,,,,,,,,
Seventh_Son,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
Welcome_to_Me,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
The_Judge,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
Transformers__Age_of_Extinction,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
The_Normal_Heart,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.051657,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
The_Phantom_Tollbooth,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
Andaz_Apna_Apna,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
Hotel_Transylvania,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.041679,0.0,0.0,0.0,0.0
The_Matrix,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.159570,0.159570,0.000000,0.0,0.0,0.0,0.0


(95, 3283)

,24000,baby,baseball,big,doc,escort,frozen,heroes,high,huck,...,years.1,york,yorks,young,young.1,younger,youngja,zebra,zellweger,zoologists
count,95.0,95.000000,95.000000,95.000000,95.0,95.0,95.0,95.000000,95.000000,95.0,...,95.000000,95.000000,95.0,95.000000,95.000000,95.000000,95.0,95.000000,95.0,95.0
mean,0.0,0.002288,0.001108,0.000353,0.0,0.0,0.0,0.000753,0.001395,0.0,...,0.003679,0.002638,0.0,0.006366,0.006366,0.001488,0.0,0.000531,0.0,0.0
std,0.0,0.019976,0.010803,0.003444,0.0,0.0,0.0,0.007335,0.008181,0.0,...,0.013408,0.013584,0.0,0.021099,0.021099,0.008699,0.0,0.005178,0.0,0.0
min,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
25%,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
50%,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
75%,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
max,0.0,0.193490,0.105290,0.033572,0.0,0.0,0.0,0.071495,0.062724,0.0,...,0.073645,0.092014,0.0,0.159570,0.159570,0.067395,0.0,0.050467,0.0,0.0


# Preprocess Data

## Description 

Beside a short description for the audio data there is no more information on how to handle the other data. For example the runtime currently is not handles as a number but as a string(object)
Since sklearn mostly expects numerical inputs, we need to encode the data. 

For different class normally you would use one-hot-encoding, but since it's not specified let's try first the easiest approach which is Labelencoding.


### Audio Data: 
As mentiones in the paper, NaN values of the audio data are set to 0 and the average of each MFCC coefficient is calculated over all frames.





In [3]:

def pre_process_audio_data():
    df_data = df_audio_data.fillna(0.0)
    return df_data.mean(axis=1)
    
def pre_process_visual_data():
    #create columns of the two rows belonging to each movie
    df_data = df_visual_data.unstack()
    return df_data
    
    
df_audio_data_processed = pre_process_audio_data()
display(df_audio_data_processed.head(20))

df_visual_data_processed = pre_process_visual_data()
display(df_visual_data_processed.head(20))

filename       freq_coeff
Seventh_Son    0             33.737346
               1             -2.259660
               2              0.822080
               3             -0.298483
               4              0.680520
               5             -0.679905
               6              0.085080
               7             -0.249879
               8             -0.025137
               9             -0.134721
               10            -0.116094
               11            -0.098648
               12             0.066234
Welcome_to_Me  0             39.561047
               1             -4.593651
               2             -0.709224
               3             -1.020713
               4              0.160524
               5              0.001964
               6             -1.487054
dtype: float64

0                   1              \
vis_data                                0         1         0         1   
filename                                                                  
Seventh_Son                      0.047044  0.056526  0.116190  0.125160   
Welcome_to_Me                    0.307170  0.304660  0.334220  0.331930   
The_Judge                        0.000000  0.000000  0.000000  0.000000   
Transformers__Age_of_Extinction  0.199960  0.189130  0.269340  0.257380   
The_Normal_Heart                 0.000000  0.000000  0.000000  0.000000   
The_Phantom_Tollbooth            0.016911  0.068953  0.014269  0.099843   
Andaz_Apna_Apna                  0.000000  0.000000  0.294160  0.294520   
Hotel_Transylvania               0.000000  0.000000  0.000000  0.000000   
The_Matrix                       0.000000  0.114860  0.000000  0.208470   
Into_the_Wild                    0.000000  0.249980  0.000000  0.322560   
Jerry_Maguire                    0.000000  0.223340  0.000000  0.278790   
Lord_of_the_Flies                0.000000  0.000000  0.000000  0.000000   
The_Great_Gatsby                 0.000000  0.000000  0.000000  0.000000   
Inherent_Vice                    0.250990  0.252360  0.338650  0.348170   
Donnie_Darko                     0.000000  0.274540  0.000000  0.283910   
Anna_Karenina                    0.000000  0.000000  0.000000  0.000000   
American_Gangster                0.231640  0.231640  0.286290  0.286290   
Full_Metal_Jacket                0.179100  0.204410  0.167630  0.163100   
Words_and_Pictures               0.267330  0.279100  0.318770  0.326630   
John_Wick                        0.256280  0.089201  0.256200  0.122960   

                                      2                   3              \
vis_data                                0         1         0         1   
filename                                                                  
Seventh_Son                      0.136330  0.146280  0.066194  0.082497   
Welcome_to_Me                    0.331120  0.331240  0.331240  0.331380   
The_Judge                        0.000000  0.000000  0.000000  0.000000   
Transformers__Age_of_Extinction  0.279860  0.274650  0.237250  0.236640   
The_Normal_Heart                 0.000000  0.000000  0.000000  0.000000   
The_Phantom_Tollbooth            0.016807  0.121550  0.031862  0.171750   
Andaz_Apna_Apna                  0.290070  0.290400  0.011351  0.011381   
Hotel_Transylvania               0.000000  0.000000  0.000000  0.000000   
The_Matrix                       0.000000  0.224650  0.000000  0.245250   
Into_the_Wild                    0.000000  0.312730  0.000000  0.243740   
Jerry_Maguire                    0.000000  0.281220  0.000000  0.253180   
Lord_of_the_Flies                0.000000  0.000000  0.000000  0.000000   
The_Great_Gatsby                 0.000000  0.000000  0.000000  0.000000   
Inherent_Vice                    0.294950  0.307040  0.183030  0.189490   
Donnie_Darko                     0.000000  0.278040  0.000000  0.300110   
Anna_Karenina                    0.000000  0.000000  0.000000  0.000000   
American_Gangster                0.300680  0.300680  0.281180  0.281180   
Full_Metal_Jacket                0.092571  0.087838  0.159180  0.143390   
Words_and_Pictures               0.308140  0.309190  0.252550  0.247950   
John_Wick                        0.235420  0.171150  0.283070  0.210950   

                                      4                ...           821  \
vis_data                                0         1    ...             0   
filename                                               ...                 
Seventh_Son                      0.072554  0.079331    ...      2.950400   
Welcome_to_Me                    0.311140  0.307880    ...     21.189000   
The_Judge                        0.000000  0.000000    ...      0.002466   
Transformers__Age_of_Extinction  0.308440  0.303320    ...     14.779000   
The_Normal_Heart                 0.038749  0.201350   

# Define Models

## Description 
These are the models described in the paper. It is not allways clear which exact models they used. (see comments)

In [4]:
from  sklearn.neighbors import KNeighborsClassifier, NearestCentroid #(not sure if this is the nearest mean classifiert) 
from  sklearn.tree import DecisionTreeClassifier
from  sklearn.linear_model import LogisticRegression
from  sklearn.svm import SVC #(not clear which SVC, there is also NuSVC )
from  sklearn.ensemble import BaggingClassifier
from  sklearn.ensemble import AdaBoostClassifier
from  sklearn.ensemble import GradientBoostingClassifier
from  sklearn.ensemble import RandomForestClassifier
from  sklearn.naive_bayes import GaussianNB # there are 3 different naive bayes classifiers, it is not stated which one they used 


model_list = [KNeighborsClassifier(),
                    DecisionTreeClassifier(),
                    LogisticRegression(),
                    SVC(),
                    BaggingClassifier(),
                    AdaBoostClassifier(),
                    GradientBoostingClassifier(),
                    RandomForestClassifier(),
                    GaussianNB() 
                   ]

    


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Define Performance measures:

As mentioned in the paper the performant measueres are the following Precision and Recall and F1-Score. To be more precise the weighted average of Precision and Recall and F1-Score as stated in the dataset paper. 

In [5]:
from sklearn.model_selection import cross_validate

def calculate_metrics(clf,X,y ):
    metric =  cross_validate(clf, X, y, scoring=('precision_weighted','recall_weighted','f1_weighted'), return_train_score=False, cv=10)  
    return pd.Series({'precision':metric['test_precision_weighted'].mean(),'recall':metric['test_recall_weighted'].mean(),'F1':metric['test_f1_weighted'].mean() })

# Select Models

As defined in the paper they use 10-fold CV on the classifiers for training and keep all the classifiers where the metrics are above 0.5 for later stacking.


In [6]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

class MultiColumnLabelEncoder:
    
    def __init__(self, columns = None):
        self.columns = columns # list of column to encode

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        
        output = X.copy()
        
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname, col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        
        return output

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
    
def getModelName( object ): 

    if hasattr(object, '__module__') and hasattr(object, '__name__'):
        return  object.__name__
    elif hasattr(object, '__module__') and hasattr(object, '__class__'):
        return  object.__class__.__name__
    else:
        raise TypeError("Could not get name of object!")
    
def evaluate_models( X, y ):
    metrics = pd.DataFrame()

    for model in model_list:

        m = calculate_metrics(model,X,y )
        metrics[getModelName(model)] = m

    return metrics.T


df_final_results = pd.DataFrame()

import warnings
warnings.filterwarnings('ignore')

In [7]:
    
df_train = pd.merge(df_labled_movies,df_meta_data, on='filename')
df_train.drop(['movie', 'filename'],axis=1, inplace=True)
display(df_train.head(2))
df_X = df_train.drop('goodforairplane',axis=1)
df_y = df_train['goodforairplane']



display("----  Lable encoded ----")
label_encoder = MultiColumnLabelEncoder(['language','year','genre','country','runtime','rated'])    
X_labelencoded = label_encoder.fit_transform(df_X)
metrics = evaluate_models(X_labelencoded, df_y)
display(metrics)

#convert runtime and year to actual number
df_X['runtime'] = df_X['runtime'].apply(lambda x: int(x.split(' ')[0]) )
df_X['year'] =  df_X['year'].apply(pd.to_numeric)

display("---- Lable encoded with float for year and runtime ----")
##optimizing encoding
label_encoder = MultiColumnLabelEncoder(['language','year','genre','country','rated'])    
X_labelencoded = label_encoder.fit_transform(df_X)
metrics = evaluate_models(X_labelencoded, df_y)
display(metrics)

display("---- Lable encoded without year ----")
label_encoder = MultiColumnLabelEncoder(['language','genre','country','rated'])    
X_labelencoded = label_encoder.fit_transform(df_X)
metrics = evaluate_models(X_labelencoded, df_y)
display(metrics)

# save the best of the for the final table 
metrics['Modality'] = 'metadata'
df_final_results = df_final_results.append(metrics)


display("---- OneHot Encoding ----")
##optimizing encoding further
X_onehotencoded = pd.get_dummies(df_X)
metrics = evaluate_models(X_onehotencoded, df_y)
display(metrics)


,goodforairplane,language,year,genre,country,runtime,rated
0,1,English,2014,"Action, Adventure, Fantasy","USA, UK, Canada, China",102 min,PG-13
1,0,English,2014,"Comedy, Drama",USA,105 min,R


'----  Lable encoded ----'

,precision,recall,F1
KNeighborsClassifier,0.519400,0.566162,0.523350
DecisionTreeClassifier,0.523778,0.519798,0.497971
LogisticRegression,0.587637,0.585556,0.574014
SVC,0.297467,0.536869,0.382622
BaggingClassifier,0.547209,0.515758,0.480662
AdaBoostClassifier,0.516002,0.501717,0.490721
GradientBoostingClassifier,0.512493,0.511818,0.498450
RandomForestClassifier,0.539624,0.517778,0.503076
GaussianNB,0.474880,0.506667,0.481515


'---- Lable encoded with float for year and runtime ----'

,precision,recall,F1
KNeighborsClassifier,0.657535,0.618586,0.600698
DecisionTreeClassifier,0.382621,0.399596,0.380096
LogisticRegression,0.512315,0.524040,0.493776
SVC,0.300554,0.547980,0.388116
BaggingClassifier,0.542088,0.517778,0.507261
AdaBoostClassifier,0.466827,0.474444,0.462241
GradientBoostingClassifier,0.430951,0.432020,0.415558
RandomForestClassifier,0.504539,0.503737,0.484702
GaussianNB,0.467194,0.499798,0.474040


'---- Lable encoded without year ----'

,precision,recall,F1
KNeighborsClassifier,0.632037,0.619697,0.589233
DecisionTreeClassifier,0.432352,0.442020,0.419286
LogisticRegression,0.545546,0.549293,0.529509
SVC,0.300554,0.547980,0.388116
BaggingClassifier,0.394108,0.407980,0.383867
AdaBoostClassifier,0.466827,0.474444,0.462241
GradientBoostingClassifier,0.424838,0.430000,0.412427
RandomForestClassifier,0.546888,0.533939,0.525844
GaussianNB,0.517396,0.539798,0.499798


'---- OneHot Encoding ----'

,precision,recall,F1
KNeighborsClassifier,0.574384,0.554242,0.544317
DecisionTreeClassifier,0.488396,0.478485,0.459617
LogisticRegression,0.335283,0.402727,0.359249
SVC,0.389456,0.453333,0.413346
BaggingClassifier,0.521665,0.503535,0.493322
AdaBoostClassifier,0.375063,0.382727,0.371664
GradientBoostingClassifier,0.340226,0.400707,0.360060
RandomForestClassifier,0.408165,0.432020,0.413088
GaussianNB,0.367316,0.400707,0.347878


In [8]:
from sklearn.preprocessing import Normalizer

################## Use textual data  ###################
display('################## Use textual data  ###################')

df_movies = df_labled_movies.drop(['movie'],axis=1)
df_train = pd.merge(df_movies,df_text_data, on='filename')
df_train.drop(['filename'],axis=1, inplace=True)
display(df_train.head(2))
df_X = df_train.drop('goodforairplane',axis=1)
df_y = df_train['goodforairplane']


display("---- RAW Data ----")
metrics = evaluate_models(df_X, df_y)
display(metrics)

# save  the final table 
metrics['Modality'] = 'textual'
df_final_results = df_final_results.append(metrics)

display("---- Normalize Data ----")
df_normalized_X = Normalizer().fit_transform(df_X)
metrics = evaluate_models(df_normalized_X, df_y)
display(metrics)




'################## Use textual data  ###################'

,goodforairplane,24000,baby,baseball,big,doc,escort,frozen,heroes,high,...,years.1,york,yorks,young,young.1,younger,youngja,zebra,zellweger,zoologists
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


'---- RAW Data ----'

,precision,recall,F1
KNeighborsClassifier,0.341994,0.465152,0.366055
DecisionTreeClassifier,0.442484,0.482424,0.443300
LogisticRegression,0.300554,0.547980,0.388116
SVC,0.300554,0.547980,0.388116
BaggingClassifier,0.508149,0.565152,0.517855
AdaBoostClassifier,0.507167,0.521717,0.498766
GradientBoostingClassifier,0.649890,0.653939,0.597817
RandomForestClassifier,0.478306,0.543131,0.490471
GaussianNB,0.537073,0.558182,0.538881


'---- Normalize Data ----'

,precision,recall,F1
KNeighborsClassifier,0.506534,0.533030,0.512809
DecisionTreeClassifier,0.486296,0.495354,0.459348
LogisticRegression,0.300554,0.547980,0.388116
SVC,0.300554,0.547980,0.388116
BaggingClassifier,0.492177,0.519596,0.477972
AdaBoostClassifier,0.477032,0.510404,0.478507
GradientBoostingClassifier,0.600671,0.620606,0.568570
RandomForestClassifier,0.409067,0.458182,0.415903
GaussianNB,0.537073,0.558182,0.538881


In [9]:
from sklearn.preprocessing import StandardScaler,RobustScaler

################## Use visual data  ###################
display('################## Use visual data  ###################')

df_movies = df_labled_movies.drop(['movie'],axis=1)
df_train = pd.merge(df_movies,df_visual_data_processed, on='filename')
df_train.drop(['filename'],axis=1, inplace=True)
display(df_train.head(5))
df_X = df_train.drop('goodforairplane',axis=1)
df_y = df_train['goodforairplane']


display("---- RAW Data ----")
metrics = evaluate_models(df_X, df_y)
display(metrics)

display("---- Scaled Data ----")
df_scaled_X = StandardScaler().fit_transform(df_X)
metrics = evaluate_models(df_scaled_X, df_y)
display(metrics)

# save  the final table 
metrics['Modality'] = 'visual'
df_final_results = df_final_results.append(metrics)

display("---- RobustScaler Data ----")
df_scaled_X = RobustScaler().fit_transform(df_X)
metrics = evaluate_models(df_scaled_X, df_y)
display(metrics)


'################## Use visual data  ###################'

,goodforairplane,"(0, 0)","(0, 1)","(1, 0)","(1, 1)","(2, 0)","(2, 1)","(3, 0)","(3, 1)","(4, 0)",...,"(821, 0)","(821, 1)","(822, 0)","(822, 1)","(823, 0)","(823, 1)","(824, 0)","(824, 1)","(825, 0)","(825, 1)"
0,1,0.047044,0.056526,0.11619,0.12516,0.13633,0.14628,0.066194,0.082497,0.072554,...,2.950400,3.216700,91672.0,81373.0,22207.0,21045.0,26201.0,24225.0,14542.0,13529.0
1,0,0.307170,0.304660,0.33422,0.33193,0.33112,0.33124,0.331240,0.331380,0.311140,...,21.189000,21.182000,81665.0,83171.0,13672.0,13714.0,32531.0,32774.0,13753.0,13780.0
2,0,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.002466,0.002466,729320.0,729320.0,119950.0,119950.0,230400.0,230400.0,119950.0,119950.0
3,0,0.199960,0.189130,0.26934,0.25738,0.27986,0.27465,0.237250,0.236640,0.308440,...,14.779000,14.723000,208630.0,211630.0,23968.0,24019.0,47979.0,48339.0,24059.0,24090.0
4,1,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.038749,...,7.379800,11.306000,145760.0,79962.0,20730.0,20617.0,35320.0,45216.0,20831.0,20738.0


'---- RAW Data ----'

,precision,recall,F1
KNeighborsClassifier,0.546787,0.542929,0.529648
DecisionTreeClassifier,0.461554,0.477980,0.463189
LogisticRegression,0.571573,0.585051,0.560193
SVC,0.398547,0.557980,0.422500
BaggingClassifier,0.553917,0.543737,0.535883
AdaBoostClassifier,0.505459,0.509293,0.496069
GradientBoostingClassifier,0.521322,0.521515,0.512374
RandomForestClassifier,0.467618,0.479091,0.465573
GaussianNB,0.503584,0.518586,0.484453


'---- Scaled Data ----'

,precision,recall,F1
KNeighborsClassifier,0.570299,0.560000,0.538146
DecisionTreeClassifier,0.576772,0.572828,0.558449
LogisticRegression,0.580084,0.549798,0.538425
SVC,0.453070,0.538889,0.448757
BaggingClassifier,0.618066,0.597273,0.590280
AdaBoostClassifier,0.507985,0.504242,0.489265
GradientBoostingClassifier,0.469628,0.490202,0.474537
RandomForestClassifier,0.561159,0.542626,0.536601
GaussianNB,0.607893,0.587273,0.573590


'---- RobustScaler Data ----'

,precision,recall,F1
KNeighborsClassifier,0.558343,0.560000,0.544837
DecisionTreeClassifier,0.523867,0.522424,0.508063
LogisticRegression,0.513166,0.509394,0.501544
SVC,0.425366,0.467172,0.420111
BaggingClassifier,0.562226,0.550808,0.546398
AdaBoostClassifier,0.497985,0.493131,0.478154
GradientBoostingClassifier,0.471742,0.492222,0.477334
RandomForestClassifier,0.476928,0.471818,0.454637
GaussianNB,0.532297,0.525354,0.522063


In [10]:
from sklearn.preprocessing import StandardScaler,RobustScaler

################## Use audio data  ###################
display('################## Use audio data  ###################')

df_movies = df_labled_movies.drop(['movie'],axis=1)
df_train = pd.merge(df_movies,pd.DataFrame(df_audio_data_processed), on='filename')
df_train.drop(['filename'],axis=1, inplace=True)
display(df_train.head(5))
df_X = df_train.drop('goodforairplane',axis=1)
df_y = df_train['goodforairplane']


display("---- RAW Data ----")
metrics = evaluate_models(df_X, df_y)
display(metrics)

# save  the final table 
metrics['Modality'] = 'audio'
df_final_results = df_final_results.append(metrics)

display("---- Scaled Data ----")
df_scaled_X = StandardScaler().fit_transform(df_X)
metrics = evaluate_models(df_scaled_X, df_y)
display(metrics)

display("---- RobustScaler Data ----")
df_scaled_X = RobustScaler().fit_transform(df_X)
metrics = evaluate_models(df_scaled_X, df_y)
display(metrics)

'################## Use audio data  ###################'

,goodforairplane,0
0,1,33.737346
1,1,-2.259660
2,1,0.822080
3,1,-0.298483
4,1,0.680520


'---- RAW Data ----'

,precision,recall,F1
KNeighborsClassifier,0.506285,0.510045,0.506474
DecisionTreeClassifier,0.543862,0.542441,0.541880
LogisticRegression,0.485729,0.541660,0.401713
SVC,0.544320,0.545673,0.416338
BaggingClassifier,0.535994,0.533504,0.533682
AdaBoostClassifier,0.524882,0.539982,0.486366
GradientBoostingClassifier,0.500367,0.520509,0.480814
RandomForestClassifier,0.535649,0.531957,0.532033
GaussianNB,0.523032,0.546532,0.431976


'---- Scaled Data ----'

,precision,recall,F1
KNeighborsClassifier,0.506285,0.510045,0.506474
DecisionTreeClassifier,0.543862,0.542441,0.541880
LogisticRegression,0.485729,0.541660,0.401713
SVC,0.486108,0.544912,0.403628
BaggingClassifier,0.546418,0.543300,0.543256
AdaBoostClassifier,0.524882,0.539982,0.486366
GradientBoostingClassifier,0.500367,0.520509,0.480814
RandomForestClassifier,0.535934,0.531917,0.531996
GaussianNB,0.523032,0.546532,0.431976


'---- RobustScaler Data ----'

,precision,recall,F1
KNeighborsClassifier,0.506285,0.510045,0.506474
DecisionTreeClassifier,0.543862,0.542441,0.541880
LogisticRegression,0.485729,0.541660,0.401713
SVC,0.505053,0.529445,0.420454
BaggingClassifier,0.536228,0.532730,0.532796
AdaBoostClassifier,0.524882,0.539982,0.486366
GradientBoostingClassifier,0.500367,0.520509,0.480814
RandomForestClassifier,0.525148,0.522200,0.522393
GaussianNB,0.523032,0.546532,0.431976


## Final base classifier filter

In [11]:
df_r = df_final_results
df_r = df_r[ (df_r['precision'] > 0.5) & (df_r['recall'] > 0.5) & (df_r['F1'] > 0.5) ]
display(df_r)


,precision,recall,F1,Modality
KNeighborsClassifier,0.632037,0.619697,0.589233,metadata
LogisticRegression,0.545546,0.549293,0.529509,metadata
RandomForestClassifier,0.546888,0.533939,0.525844,metadata
BaggingClassifier,0.508149,0.565152,0.517855,textual
GradientBoostingClassifier,0.649890,0.653939,0.597817,textual
GaussianNB,0.537073,0.558182,0.538881,textual
KNeighborsClassifier,0.570299,0.560000,0.538146,visual
DecisionTreeClassifier,0.576772,0.572828,0.558449,visual
LogisticRegression,0.580084,0.549798,0.538425,visual
BaggingClassifier,0.618066,0.597273,0.590280,visual


As we can see the results table looks pretty different than in the paper. There is not really enough information in the paper to be sure that we are correctly reproducing the steps. 

With the audio data there is actually not really more we could do since we just end up with one coliumn of data as descirbed in the paper, but still the metrics is not as good as in the paper. 

__Is there something wrong already when we load the data ? Wrong data?__

